In [1]:
import pandas as pd
import numpy as np
from snowflake.sqlalchemy import URL
import datetime as datetime
import warnings
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

/Users/keyurpethad/anaconda3/lib/python3.11/site-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
month = "Dec'23"
prev_month = "Nov'23"
two_months_ago = "Oct'23"
three_months_ago = "Sep'23"

In [3]:
master_calling = pd.read_excel("PL Loan Book Jan'24.xlsx", header=2)

In [4]:
master_calling.head()

,niro_user_id,Niro Opportunity ID,Disbursal Partner Name,Platform Partner Name,Lender Loan Id,Disbursement Month,Month Number,Disbursement Year,Phone Numbers,Disbursement Date,Presentation Lag,Gross PF Charged,PF Refund,Final Proc Fees,GST,Pre-EMI,Disbursed Amount,Loan Amount (all inclusive),Loan Amount Rank,Interest Rate,Tenor In Months,Tenor Rank,PF %,Cost of Funds,Cost of Funds (New),Spread,Monthly Emi,Monthly Income (Net Take Home),Income Rank,Income source,Self Employed,Nature of Business,# Years in Employment,Residence Ownership Type,Owned,Purpose of Loan,CIBIL Score,CIBIL Band,CIBIL Score Rank,Risk Band,Risk Segmentation,Disbursement TAT,City,State,Pin Code,Tier,Gender,Male,Female,Age,customer_router,CPV Initiated,Lead Name,Loan Status,Foreclosure Date,Foreclosure Details,NPA Details,NPA setteled Status,Write-off Month,Feb Presentation,Mar Presentation,Apr Presentation,May Presentation,Jun Presentation,Jul Presentation,Aug Presentation,Sep Presentation,Oct Presentation,Nov Presentation,Dec Presentation,Jan'23 Presentation,Feb'23 Presentation,Mar'23 Presentation,Apr'23 Presentation,May'23 Presentation,Jun'23 Presentation,July'23 Presentation,Aug'23 Presentation,Sep'23 Presentation,Oct'23 Presentation,Nov'23 Presentation,Dec'23 Presentation,Jan'24 Presentation,Feb DPD,Mar DPD,Apr DPD,May DPD,Jun DPD,Jul DPD,Aug DPD,Sep DPD,Oct DPD,Nov DPD,Dec DPD,Jan'23 DPD,Feb'23 DPD,Mar'23 DPD,Apr'23 DPD,May'23 DPD,Jun'23 DPD,July'23 DPD,Aug'23 DPD,Sep'23 DPD,Oct'23 DPD,Nov'23 DPD,Dec'23 DPD,Mar DPD Rank,PayU'July'23 DPD,PayU'Aug'23 DPD,PayU'Sep'23 DPD,PayU'Oct'23 DPD,PayU'Nov'23 DPD,PayU'Dec'23 DPD,Sep DPD Rank,Jan - POS,Feb - POS,Mar - POS,Apr - POS,May - POS,Jun - POS,Jul - POS,Aug - POS,Sep - POS,Oct - POS,Nov - POS,Dec - POS,Jan'23 - POS,Feb'23 - POS,Mar'23 - POS,Apr'23 - POS,May'23 - POS,Jun'23 - POS,July'23 - POS,Aug'23 - POS,Sep'23 - POS,Oct'23 - POS,Nov'23 - POS,Dec'23 - POS,Int * Loan Amount (all Incl),CoF * Loan Amount (all Incl),Tenor * Loan Amount (all Incl),PF * Loan Amount (all Incl),Spread * Loan Amount (all Incl),FOIR,Ok/Problem Areas,Repeat Cases,DPD 90+ Months,DPD 60+ Months,90+ Amount,Feb - Repaid,Mar - Repaid,Apr - Repaid,May - Repaid,Jun - Repaid,Jul - Repaid,Aug - Repaid,Sep - Repaid,Oct - Repaid,Nov - Repaid,Dec - Repaid,Jan'23 - Repaid,Feb'23 - Repaid,Mar'23 - Repaid,Apr'23 - Repaid,May'23 - Repaid,Jun'23 - Repaid,July'23 - Repaid,Aug'23 - Repaid,Sep'23 - Repaid,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,0f100756-11ae-4472-bd29-4fdbd6c37fdb,liquiloans,quikr,499217,2021-12-31,1,2021,9900368067,2021-12-31,Yes,2366,0,2366,426,0.0,50005,52797,2,27.0,12,2,0.044813,17.4,17.4,9.6,5069,26000,3,Salaried,0,'-',1,Rented,0,Others,764,760+,8,CAT-D,True,< 24 hrs,Bangalore,KARNATAKA,560085,Tier 1,Female,0,1,39,PL_OLD_PREAPPROVED,Yes,VINUTHA HONNE GOWDA,Closed,NaT,NaN,NaN,No,-,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Cleared,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-,-,-,-,NaN,-,-,-,-,-,-,NaN,52797.0,48915.93250,44947.540981,40889.860653,36740.882518,32498.552375,28160.769803,23722.160542,19190.208334,14553.208334,9811.208334,4962.960521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1425519.0,918667.8,633564,2366.0,506851.2,0.44,OK Area,1.0,-,-,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31,1.0
1,e85b616e-e54b-4d67-b929-5f4b612404e2,188ec580-5b20-435b-8b02-b1fd0c439c6b,liquiloans,quikr,500342,2022-01-31,2,2022,8765513171,2022-01-04,No,2366,0,2366,426,0.0,50005,52797,2,27.0,12,2,0.044813,17.4,17.4,9.6,5069,20000,2,Salaried,0,'-',1.5,Owned,1,Marriage,746,740-760,7,CAT-D,True,< 24 hrs,Lucknow,UTTAR PRADESH,226029,Tier 2,Male,1,0,37,PL_OLD_PREAPPROVED,Yes,AKHILESH KUMAR YADAV,Foreclosed,2022-09-12,NaN,NaN,No,-,Cleared,Cleared,Cleared,Cleared,BOUNCED,BOUNCED,Bounce

In [5]:
#State
#City
#Tier (city tier)
#Repeat Cases (TopUps?)

#### Pre disbursal data

In [6]:
from sqlalchemy import create_engine


class extractorFromQuery:
    def __init__(self):
        self.credentials = {
            "account": "qfi-niro",
            "user": "niro_analytics",
            "password": "4qVe_V-aEq,j.i,",
            "warehouse": "NIRO_ANALYTICS_WAREHOUSE",
        }

    def queryExecutor(self, query: str, *args):
        engine = create_engine(URL(**self.credentials))
        res = pd.read_sql_query(query, engine, params=args)
        return res

In [7]:
disbursed_cases = """
    select 
    la.id,
    ou.category,
    la.principal_amount,
    la.interest_rate,
    la.tenure,
    o.attributed_partner as "supply",
    o.propensity,
    o.offer_type,
    tu.income_source,
    tu.purpose_of_loan,
    tu.monthly_take_home_income,
    tu.residence_ownership_type,
    tu.marital_status,
    cpv.cpv_type,
    l.name as "demand",
    CASE
        when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
        else date(la.loan_disbursed_date_by_ops)
    end as disbursement_date
    from niro_rds.niro_data.core__public__loan_application la
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__OFFERS o on o.id = la.offer_id
    left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = o.id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__VERIFICATION_DETAILS vd on vd.id = la.verification_id 
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__TELE_PD_USER tu on vd.telepd_id::bigint = tu.id
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__CP_VERIFICATION cpv on cpv.id = vd.cpv_id::bigint
    left join NIRO_RDS.NIRO_DATA.CORE__PUBLIC__LENDERS l on l.id = o.lender_uid
    where la.loan_application_status in ('CLOSED','LOAN_EXECUTED') and la.is_deleted = False and o.is_deleted = False;
"""
total_disbursed_cases = extractorFromQuery().queryExecutor(disbursed_cases)
total_disbursed_cases.shape

(43322, 19)

In [16]:
total_disbursed_cases['residence_ownership_type'] = total_disbursed_cases.residence_ownership_type.str.lower().replace({'family_owned':'family owned'}).replace({'owned':'self owned'}).replace({'pg':'rented'}).replace({'room sharing':'rented'}).replace({'ancestral property':'family owned'})

residence_ownership_type
family owned            23322
rented                  12379
self owned               5319
office accommodation      945
others                    578
relative owned            266
Name: count, dtype: int64

In [21]:
disbursed_cases_with_cvattrs = """
            select * from (
            select 
            distinct la.id,
            pii.gender as bureau_gender,
            pii.pin as bureau_pincode,
            pii.calculated_age,
            pii.city as bureau_city,
            pii.state as bureau_state,
            pii.native_language,
            date_trunc("MONTH",date(od.created_at)) as createdate,
            od.current_salary,
            od.available_income,
            od.foir_dlq, od.foir_cibil, od.foir_trd, od.foir_mean, od.new_foir, 
            od.naps_score as naps_disb_score,
            rank()over(partition by od.user_id order by od.created_at asc) as rnk,
            CASE
                when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
                else date(la.loan_disbursed_date_by_ops)
            END as disbursement_date
            from NIRO_RDS.NIRO_DATA.CORE__PUBLIC__LOAN_APPLICATION la
            left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = la.offer_id
            left join ANALYTICS.OFFERS.offer_details od on od.user_id = la.user_id and od.category = ou.category
            left join ANALYTICS.OFFERS.personal_info pii on od.pii_id = pii.pii_id
            where la.loan_application_status in ('CLOSED','LOAN_EXECUTED') 
            and la.is_deleted = False
            and(la.is_deleted_at_source = false or la.is_deleted_at_source is null)
            and od.dec_reason = 'NOT DECLINED')
where rnk = 1;
"""
disbursed_cases = (
    extractorFromQuery()
    .queryExecutor(disbursed_cases_with_cvattrs)
    .drop(columns="disbursement_date")
)
disbursed_cases.shape

(41603, 17)

In [24]:
disbursed_cases = disbursed_cases.drop_duplicates(subset=["id"])
predisbursal_data = total_disbursed_cases.merge(disbursed_cases, on="id", how="left").rename(
    columns={"id": "niro_opportunity_id"}
)
predisbursal_data.drop_duplicates(subset=["niro_opportunity_id"], inplace=True)
predisbursal_data.shape

(43322, 35)

In [28]:
predisbursal_data['propensity'] = np.where(predisbursal_data['propensity'].isin(['HiiProp', 'High']), 'High',
                                  np.where(predisbursal_data['propensity'].isin(['MedProp', 'Med', 'Medium']), 'Medium',
                                  np.where(predisbursal_data['propensity'].isin(['LowProp', 'Low']), 'Low',None)))

propensity
High    27860
None    11609
Low      3853
Name: count, dtype: int64

In [25]:
predisbursal_data.head()

,niro_opportunity_id,category,user_id,principal_amount,interest_rate,tenure,supply,propensity,offer_type,customer_availability,income_source,purpose_of_loan,monthly_take_home_income,residence_ownership_type,marital_status,cpv_type,agency_status,demand,disbursement_date,bureau_gender,bureau_pincode,calculated_age,bureau_city,bureau_state,native_language,createdate,current_salary,available_income,foir_dlq,foir_cibil,foir_trd,foir_mean,new_foir,naps_disb_score,rnk
0,296c3df2-9f75-47ec-8705-27d5c004386e,CAT-B,ce3e2986-173e-45f4-97ec-4647742bbd24,214116,27,24,finbud,HiiProp,PRE_APPROVED,Yes,Salaried,House Renovation,21000,family owned,MARRIED,REMOTE,Positive,liquiloans,2023-04-24,Female,673008,52.0,KOZHIKODE,KERALA,English,2023-02-01,56250.00,21265.60,0.5,0.4,0.50,0.47,71.78,703.0,1.0
1,0925d1c4-5371-403f-b519-ca1ac4b18bea,CAT-C,3c100d33-4e71-413a-92f3-60472c97dded,177823,21,18,finbud,HiiProp,PRE_APPROVED,Yes,Salaried,House Renovation,38000,family owned,MARRIED,REMOTE,Positive,payu,2022-11-20,Female,562114,51.0,CHIKKABALLAPUR,KARNATAKA,English,2022-11-01,130303.42,30042.18,0.5,0.4,0.37,0.42,62.32,720.0,1.0
2,55992f7e-3692-4488-970e-a96c6a3436bc,CAT-B,6bfecd70-4cdf-489a-bd9d-4ef2b11bef50,137195,20,12,snapdeal,MedProp,PRE_APPROVED,Yes,Salaried,Purchase,45000,family owned,MARRIED,REMOTE,Positive,payu,2023-08-07,Male,500049,33.0,HYDERABAD,ANDHRA PRADESH,Tamil,2023-06-01,317160.00,136754.85,0.5,0.4,0.36,0.42,27.64,816.0,1.0
3,187aa56e-c9c7-4fda-8796-14d08feaa5aa,CAT-B,0db6e178-c690-43ad-8f7b-7c83fbef64c9,393004,19,36,snapdeal,HiiProp,PRE_APPROVED,Yes,Salaried,Down Payment,30500,self owned,MARRIED,REMOTE,Positive,payu,2022-11-19,Male,382015,38.0,GANDHI NAGAR,GUJARAT,None,2022-10-01,274860.00,140636.90,0.5,0.4,0.43,0.44,19.34,761.0,1.0
4,0a80554c-a906-4af0-a10c-1d2f4cf31b2a,CAT-C,afa235e1-d716-448e-97a5-56963677c69a,167718,21,18,niro,HiiProp,PRE_APPROVED,Yes,Salaried,Health/Hospital,30150,family owned,MARRIED,PHYSICAL,Positive,payu,2022-06-04,Male,462027,27.0,BHOPAL,MADHYA PRADESH,None,2022-05-01,48000.00,23477.50,0.5,0.4,0.45,0.45,65.56,738.0,1.0


#### EMI date Clearance type

In [ ]:
master_calling[month + " clearance status"] = np.where(master_calling[month + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[month + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[month + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[month + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[month + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[month + " Presentation"])))))

master_calling[prev_month + " clearance status"] = np.where(master_calling[prev_month + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[prev_month + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[prev_month + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[prev_month + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[prev_month + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[prev_month + " Presentation"])))))
                                                
master_calling[two_months_ago + " clearance status"] = np.where(master_calling[two_months_ago + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[two_months_ago + " Presentation"])))))

master_calling[three_months_ago + " clearance status"] = np.where(master_calling[three_months_ago + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[three_months_ago + " Presentation"])))))

#### Pre presentation bucket

In [ ]:
master_calling[prev_month + " DPD"].unique()

In [ ]:
((pd.to_numeric(master_calling[prev_month + " DPD"], errors='coerce') >= 0) & (pd.to_numeric(master_calling[prev_month + " DPD"], errors='coerce') < 29)).sum()

In [ ]:
master_calling[month + " Presentation bucket"] = np.where(master_calling[prev_month + " DPD"].isin(['-']) | ((pd.to_numeric(master_calling[prev_month + " DPD"], errors='coerce') >= -45) & (pd.to_numeric(master_calling[prev_month + " DPD"], errors='coerce') <= 29)), 'A) Bkt 0',
                                                np.where(master_calling[prev_month + " DPD"].isin(['1-29+']), 'B) Bkt 1',
                                                np.where(master_calling[prev_month + " DPD"].isin(['30+']), 'C) Bkt 2',
                                                np.where(master_calling[prev_month + " DPD"].isin(['60+']), 'D) Bkt 3',
                                                np.where(master_calling[prev_month + " DPD"].isin(['90+']), 'E) Bkt 4',
                                                np.where(master_calling[prev_month + " DPD"].isin(['NPA (Unsettled)', 'NPA']), 'F) Bkt 4+',
                                                np.where(master_calling[prev_month + " DPD"].isin(['Settled', 'Closed', 'Settled (partial paid)', 'Foreclosed']), 'G) Closed',master_calling[prev_month + " DPD"])))))))

master_calling[prev_month + " Presentation bucket"] = np.where(master_calling[two_months_ago + " DPD"].isin(['-']) | ((pd.to_numeric(master_calling[two_months_ago + " DPD"], errors='coerce') >= -45) & (pd.to_numeric(master_calling[two_months_ago + " DPD"], errors='coerce') <= 29)), 'A) Bkt 0',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['1-29+']), 'B) Bkt 1',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['30+']), 'C) Bkt 2',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['60+']), 'D) Bkt 3',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['90+']), 'E) Bkt 4',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['NPA (Unsettled)', 'NPA']), 'F) Bkt 4+',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['Settled', 'Closed', 'Settled (partial paid)', 'Foreclosed']), 'G) Closed',master_calling[two_months_ago + " DPD"])))))))

master_calling[two_months_ago + " Presentation bucket"] = np.where(master_calling[three_months_ago + " DPD"].isin(['-']) | ((pd.to_numeric(master_calling[three_months_ago + " DPD"], errors='coerce') >= -45) & (pd.to_numeric(master_calling[three_months_ago + " DPD"], errors='coerce') <= 29)), 'A) Bkt 0',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['1-29+']), 'B) Bkt 1',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['30+']), 'C) Bkt 2',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['60+']), 'D) Bkt 3',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['90+']), 'E) Bkt 4',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['NPA (Unsettled)', 'NPA']), 'F) Bkt 4+',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['Settled', 'Closed', 'Settled (partial paid)', 'Foreclosed']), 'G) Closed',master_calling[three_months_ago + " DPD"])))))))


### TAT calculation

In [ ]:
prev_month_df = pd.read_excel("Repayment_data_Sep'23 to Dec'23.xlsx", sheet_name=prev_month)
two_months_ago_df = pd.read_excel("Repayment_data_Sep'23 to Dec'23.xlsx", sheet_name=two_months_ago)
three_months_ago_df = pd.read_excel("Repayment_data_Sep'23 to Dec'23.xlsx", sheet_name=three_months_ago)

In [ ]:
prev_month_df['EMI Submission date'] = pd.to_datetime(prev_month_df['EMI Submission date'])
prev_month_df['Collection Date'] = pd.to_datetime(prev_month_df['Collection Date'], errors='coerce')

two_months_ago_df['EMI Submission date'] = pd.to_datetime(two_months_ago_df['EMI Submission date'])
two_months_ago_df['Collection Date'] = pd.to_datetime(two_months_ago_df['Collection Date'], errors='coerce')

three_months_ago_df['EMI Submission date'] = pd.to_datetime(three_months_ago_df['EMI Submission date'])
three_months_ago_df['Collection Date'] = pd.to_datetime(three_months_ago_df['Collection Date'], errors='coerce')

In [ ]:
prev_month_df[prev_month + ' TAT'] = np.where(prev_month_df['Bounce Paid/unpaid'].isin(['Unpaid']), 999,
                        np.where(prev_month_df['Bounce Paid/unpaid'].isin(['Paid']),(prev_month_df['Collection Date'] - prev_month_df['EMI Submission date']).dt.days,
                        np.nan))

two_months_ago_df[two_months_ago + ' TAT'] = np.where(two_months_ago_df['Bounce Paid/unpaid'].isin(['Unpaid']), 999,
                        np.where(two_months_ago_df['Bounce Paid/unpaid'].isin(['Paid']),(two_months_ago_df['Collection Date'] - two_months_ago_df['EMI Submission date']).dt.days,
                        np.nan))

three_months_ago_df[three_months_ago + ' TAT'] = np.where(three_months_ago_df['Bounce Paid/unpaid'].isin(['Unpaid']), 999,
                        np.where(three_months_ago_df['Bounce Paid/unpaid'].isin(['Paid']),(three_months_ago_df['Collection Date'] - three_months_ago_df['EMI Submission date']).dt.days,
                        np.nan))

prev_month_df[prev_month + ' TAT_bkt'] = np.where(prev_month_df[prev_month + ' TAT'] < -60, np.nan,
                                            np.where(prev_month_df[prev_month + ' TAT'] <= -10, 'A) <= -10',
                                            np.where(prev_month_df[prev_month + ' TAT'] < 0, 'B) -10 to -1',
                                            np.where(prev_month_df[prev_month + ' TAT'] == 0, 'C) 0',
                                            np.where(prev_month_df[prev_month + ' TAT'] <= 4, 'D) 1-4',
                                            np.where(prev_month_df[prev_month + ' TAT'] <= 10, 'E) 4-10',
                                            np.where(prev_month_df[prev_month + ' TAT'] <= 29, 'F) 10 - 29',
                                            np.where(prev_month_df[prev_month + ' TAT'] == 999, 'G) Unpaid',prev_month_df[prev_month + ' TAT']))))))))

two_months_ago_df[two_months_ago + ' TAT_bkt'] = np.where(two_months_ago_df[two_months_ago + ' TAT'] < -60, np.nan,
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] <= -10, 'A) <= -10',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] < 0, 'B) -10 to -1',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] == 0, 'C) 0',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] <= 4, 'D) 1-4',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] <= 10, 'E) 4-10',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] <= 29, 'F) 10 - 29',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] == 999, 'G) Unpaid',two_months_ago_df[two_months_ago + ' TAT']))))))))

three_months_ago_df[three_months_ago + ' TAT_bkt'] = np.where(three_months_ago_df[three_months_ago + ' TAT'] < -60, np.nan,
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] <= -10, 'A) <= -10',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] < 0, 'B) -10 to -1',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] == 0, 'C) 0',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] <= 4, 'D) 1-4',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] <= 10, 'E) 4-10',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] <= 29, 'F) 10 - 29',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] == 999, 'G) Unpaid',three_months_ago_df[three_months_ago + ' TAT']))))))))

In [ ]:
master_calling.head(2)

In [ ]:
prev_month_df.head(1)

In [ ]:
master_calling = master_calling.merge(prev_month_df.rename(columns = {'Opportunity ID':'Niro Opportunity ID'})[["Niro Opportunity ID", prev_month + ' TAT', prev_month + ' TAT_bkt']], on = "Niro Opportunity ID", how = "left") \
    .merge(two_months_ago_df.rename(columns = {'Opportunity ID':'Niro Opportunity ID'})[["Niro Opportunity ID", two_months_ago + ' TAT', two_months_ago + ' TAT_bkt']], on = "Niro Opportunity ID", how = "left") \
    .merge(three_months_ago_df.rename(columns = {'Opportunity ID':'Niro Opportunity ID'})[["Niro Opportunity ID", three_months_ago + ' TAT', three_months_ago + ' TAT_bkt']], on = "Niro Opportunity ID", how = "left") 

#### Advance Collection file

In [ ]:
adv_coll_tag = pd.read_excel('advance collection detailing report.xlsx', sheet_name=month).rename(columns = {'opportunity_id':'Niro Opportunity ID', 'Code':'Code_'+month, 'Status':'Status_'+month, 'Code_meaning':'Code_meaning_'+month})
adv_coll_tag.head()

In [ ]:
master_calling = master_calling.merge(adv_coll_tag[['Niro Opportunity ID', 'Code_'+month, 'Status_'+month, 'Code_meaning_'+month]], on = 'Niro Opportunity ID', how='left')

In [ ]:
master_calling.head()

In [ ]:
# master_calling.to_excel("master_file_" + month + ".xlsx", index=False)

#### Scrub data

In [29]:
scrub_data = pd.read_csv('Disbursed_cases_Scrub_05112023.csv')
scrub_data.head()

,Sequence No,Account No,Adviser Transaction No,Member Reference,Mobile_No,Total Features,AGG911,RVLR01,BCPMTSTR,CV11,CV14,MT28S,MT33S,PL33S,AT20S,MT01S,BC02S,BG01S,CV10,TRD,AT33A,AU33S,CO04S180,AU28S,PL28S,CO01S180,BC28S,CV12,CO05S,G310S,AGGS911,AT01S,AT33A_NE_CCOD,CV14_12M,CV14_6M,CV14_3M,CV14_1M,G310S_24M,G310S_6M,G310S_3M,G310S_1M,CV11_24M,CV11_12M,G057S_1DPD_36M,G057S_1DPD_12M,BC106S_60DPD,BC107S_24M,BC106S_60DPD_12M,BC107S_12M,BC106S_LE_30DPD_12M,BC09S_36M_HCSA_LE_30,PL09S_36M_HCSA_LE_30,AT09S_6M,G310S_36M,AT33A_NE_WO,AT09S_12M,AT09S_3M,CV13,CV24,REVS904,CV20,CV22,UL_TRD,CV21,G310S_2M,Secured Accounts Count,Unsecured Accounts Count,Secured High Credit Sum,Unsecured High Credit Sum,Secured Amount Overdue Sum,Unsecured Amount Overdue Sum,Secured Balances Sum,Unsecured Balances Sum,Own Accounts count,Other Accounts count,CIBILTUSC3 Score Value,CIBILTUSC3 Score Reason Code Set,CIBILTUSC3 Score Exclusion Code Set,CIBILTUSC3 Score Error Code Set,Unnamed: 78
0,0,NaN,1524319715,ASGPR7783R,9914657792,59,5.56,0.00,RVLRPLUS,0,50,-1.0,-1.0,803368.0,102,0,3,0,0,21,845232.0,-1.0,-1,-1.0,873464.0,0,550000.0,0,-1.0,1.5,0,11,826686.0,24,7,3,0,1.5,1.5,1.5,0.0,0,0,21,21,0,0,0,0,6,2,0,0,1.5,-1.0,4,0,0,15000.0,138641.0,927.30,18546.0,20,15000.0,1.5,0,14,0.0,1828480.0,0.0,11744.0,0.0,845232.0,0,15,720,"""[37, 35, 39, 26, 25]""","""[]""","""[]""",NaN
1,1,NaN,1524319716,ANJPK5437H,9398125492,59,-1.00,-1.00,NOBC,0,18,2000000.0,1028693.0,1203639.0,226,1,-1,0,0,8,2232332.0,-3.0,-1,-3.0,1203639.0,0,-1.0,0,-1.0,1.0,-1,6,2232332.0,3,2,2,0,1.0,1.0,1.0,0.0,0,0,8,8,-1,-1,-1,-1,-1,-5,0,2,1.0,-1.0,2,2,0,-1.0,-1.0,4114.00,-1.0,6,418713.0,0.0,1,4,2000000.0,1900482.0,0.0,0.0,1028693.0,1203639.0,0,5,776,"""[45]""","""[]""","""[]""",NaN
2,2,NaN,1524319717,HTOPK6367Q,7305601276,59,-1.00,-1.00,NOBC,1,20,-1.0,-1.0,19970.0,66,0,-1,0,1,6,19970.0,-1.0,4,-1.0,69484.0,1,-1.0,1,-1.0,6.0,-1,5,19970.0,12,5,3,0,6.0,6.0,6.0,0.0,1,1,6,6,-1,-1,-1,-1,-1,-5,2,0,6.0,-1.0,2,0,17,-1.0,-1.0,5749.00,-1.0,5,58843.0,6.0,0,5,0.0,84224.0,0.0,22593.0,0.0,19970.0,0,5,659,"""[03, 08, 02, 07, 04]""","""[]""","""[]""",NaN
3,3,NaN,1524319720,KVDPK8164F,9423516766,59,13.73,0.00,TRANSACTOR,0,10,-1.0,-1.0,26407.0,48,0,1,0,0,16,104847.0,-1.0,-1,-1.0,52412.0,0,15000.0,0,-1.0,1.5,14,10,102787.0,0,0,0,0,1.5,1.5,1.5,0.0,0,0,16,16,0,0,0,0,1,1,5,6,1.5,-1.0,6,3,0,-2.0,2060.0,25434.00,2060.0,8,75222.0,1.0,8,8,127609.0,108971.0,0.0,0.0,58985.0,45862.0,0,16,708,"""[35, 40, 39, 26, 38]""","""[]""","""[]""",NaN
4,4,NaN,1524319721,LBVPS4263G,8077030887,59,71.26,22.28,RVLRPLUS,0,18,-1.0,-1.0,236685.0,37,0,5,0,0,16,309375.0,-1.0,-1,-1.0,279385.0,0,205205.0,0,-1.0,1.0,21,9,272370.0,12,6,2,0,1.0,1.0,1.0,0.0,0,0,16,16,0,0,0,0,5,5,1,3,1.0,-1.0,10,1,0,30349.0,32274.0,15498.25,37005.0,14,55219.0,1.0,0,14,0.0,519439.0,0.0,0.0,0.0,309375.0,0,16,767,"""[39, 25, 36, 40, 38]""","""[]""","""[]""",NaN


In [ ]:
column_name_map = {'AGG911': 'max_aggregate_bankcard_utilisation_l12m',
 'RVLR01': 'cc_util_revolving_l1m',
 'BCPMTSTR': 'bank_card_payment_category',
 'CV11': 'no_of_60p_accs_ever',
 'CV14': 'no_of_deduped_inquiries',
 'MT28S': 'total_cl_open_mortgage_trades_l12m',
 'MT33S': 'total_balance_open_mortgage_trades_l12m',
 'PL33S': 'total_bal_of_open_pl_l12m',
 'AT20S': 'months_since_oldest_trade',
 'MT01S': 'no_of_mortgage_trades',
 'BC02S': 'no_of_open_cc_trades',
 'BG01S': 'no_of_business_general_trades',
 'CV10': 'no_of_30p_accs_ever',
 'TRD': 'no_of_trades',
 'AT33A': 'total_bal_open_trades_l12m',
 'AU33S': 'total_bal_open_auto_trades_l12m',
 'CO04S180': 'months_since_recent_chargedoff180P',
 'AU28S': 'total_cl_open_auto_trades_l12m',
 'PL28S': 'total_cl_open_pl_trades_l12m',
 'CO01S180': 'no_of_chargedoff180P',
 'BC28S': 'total_cl_of_cc_trades_l12m',
 'CV12': 'no_of_90p_accs_ever',
 'CO05S': 'total_bal_of_chargedoff_trades',
 'G310S': 'worst_rating_l12m',
 'AGGS911': 'current_utilization_of_top_wallet_bankcard',
 'AT01S': 'no_of_trades_pl_2w_gl_cd',
 'AT33A_NE_CCOD': 'total_bal_open_trades_l12m_except_wo_cc_od',
 'CV14_12M': 'no_of_deduped_inquiries_l12m',
 'CV14_6M': 'no_of_deduped_inquiries_l6m',
 'CV14_3M': 'no_of_deduped_inquiries_l3m',
 'CV14_1M': 'no_of_deduped_inquiries_l1m',
 'G310S_24M': 'worst_rating_l24m',
 'G310S_6M': 'worst_rating_l6m',
 'G310S_3M': 'worst_rating_l3m',
 'G310S_1M': 'worst_rating_l1m',
 'CV11_24M': 'no_of_60p_accs_l24m',
 'CV11_12M': 'no_of_60p_accs_l12m',
 'G057S_1DPD_36M': 'no_of_trades_bounced_l36m',
 'G057S_1DPD_12M': 'no_of_trades_bounced_l12m',
 'BC106S_60DPD': 'no_of_cc_trades_60P_l24m',
 'BC107S_24M': 'no_of_30P_cc_l24m',
 'BC106S_60DPD_12M': 'no_of_cc_trades_60P_l12m',
 'BC107S_12M': 'no_of_30P_cc_l12m',
 'BC106S_LE_30DPD_12M': 'no_of_cc_lt30p_l12m',
 'BC09S_36M_HCSA_LE_30': 'no_cc_sanctionedamt_lt30K_l36m',
 'PL09S_36M_HCSA_LE_30': 'no_pl_sanctionedamt_lt30K_l36m',
 'AT09S_6M': 'no_tr_opened_l6m',
 'G310S_36M': 'worst_dpd_l36m',
 'AT33A_NE_WO': 'total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades',
 'AT09S_12M': 'no_tr_opened_l12m',
 'AT09S_3M': 'no_tr_opened_l3m',
 'CV13': 'per_of_30p_accs_ever',
 'CV24': 'total_payment_amount_bankcard_accs_l3m',
 'REVS904': 'max_revolving_monthly_spend_l12m',
 'CV20': 'total_monthly_obligations_l3m',
 'CV22': 'total_balance_bankcard_accs_l3m',
 'UL_TRD': 'no_of_trades_unsec',
 'CV21': 'total_payment_amount_l3m',
 'G310S_2M': 'worst_dpd_l2m'}

In [ ]:
scrub_data.rename(columns = column_name_map, inplace=True)

In [ ]:
scrub_data.head()

In [ ]:
def cibil_score_bkt(row, var):
    if row[var] <= 736:
        return '<=736'
    elif row[var] <= 745:
        return '737-745'
    elif row[var] <= 749:
        return '746-749'
    elif row[var] <= 757:
        return '750-757'
    elif row[var] <= 760:
        return '758-760'
    elif row[var] <= 766:
        return '761-766'
    elif row[var] <= 772:
        return '767-772'
    elif row[var] <= 776:
        return '773-776'
    elif row[var] <= 784:
        return '777-784'
    else:
        return '785+'

def naps_score_bkt(row, var):
    if row[var] <= 689:
        return '<=689'
    elif row[var] <= 704:
        return '690-704'
    elif row[var] <= 713:
        return '705-713'
    elif row[var] <= 728:
        return '714-728'
    elif row[var] <= 742:
        return '723-742'
    elif row[var] <= 751:
        return '743-751'
    elif row[var] <= 770:
        return '752-770'
    elif row[var] <= 790:
        return '771-790'
    elif row[var] <= 818:
        return '791-818'
    else:
        return '819+'


#statename
#BCPMTSTR
scrub_data.rename(columns = {'CIBILTUSC3 Score Value':'CIBIL_score'}, inplace=True)
scrub_data['CIBIL_score_bkt'] = scrub_data.apply(cibil_score_bkt,var='CIBIL_score',axis=1)

scrub_data['no_of_open_cc_trades_bkt'] = np.select(condlist=[scrub_data['no_of_open_cc_trades']<=0,
                                             scrub_data['no_of_open_cc_trades']<=2,
                                             scrub_data['no_of_open_cc_trades']<=4,
                                             scrub_data['no_of_open_cc_trades']>4,],choicelist=['No Credit Cards','1-2',
                                                                                        '3-4','5+'],default='No Credit Cards')
scrub_data['Years on Bureau_bkt'] = np.select(condlist=[scrub_data['months_since_oldest_trade']<=24,
                                             scrub_data['months_since_oldest_trade']<=48,
                                             scrub_data['months_since_oldest_trade']<=60,
                                             scrub_data['months_since_oldest_trade']>60,],choicelist=['1-2 Yrs','2-4 Yrs',
                                                                                        '4-5 Yrs','5+ Yrs'],default='None')

scrub_data['no_of_deduped_inquiries_bkt'] = np.select(condlist=[scrub_data['no_of_deduped_inquiries_l12m']<=1,
                                             scrub_data['no_of_deduped_inquiries_l12m']<=3,
                                             scrub_data['no_of_deduped_inquiries_l12m']<=6,
                                             scrub_data['no_of_deduped_inquiries_l12m']>6,],choicelist=['<=1','2-3',
                                                                                        '4-6','7+'],default='None')

scrub_data['Secured Loan_bkt'] = np.select(condlist=[scrub_data['Secured High Credit Sum']<=0,
                                             scrub_data['Secured High Credit Sum']<=200000,
                                             scrub_data['Secured High Credit Sum']<=750000,
                                             scrub_data['Secured High Credit Sum']>750000,],choicelist=['No Secured loan','Secured Loan < 2L',
                                                                                        'Secured Loan < 7.5L','Secured Loan 7.5L+'],
                                  default='No Secured loan')

# final['Income_bkt'] = np.select(condlist=[final['ctc']<=15000,
#                                             final['ctc']<=20000,
#                                             final['ctc']<=25000,
#                                              final['ctc']<=30000,
#                                              final['ctc']<=35000,
#                                             final['ctc']<=40000,
#                                             final['ctc']<=50000,
#                                             final['ctc']<=75000,
#                                              final['ctc']>75000],choicelist=['<=15000','15000-20000','20000-25000','25000-30000',
#                                                                                 '30000-35000','35000-40000','40000-50000',
#                                                                                 '50000-75000','75000+'],\
#                             default='None')

# final['FOIR_Band'] = np.select(condlist=[final['New_FOIR']<=35,
#                                              final['New_FOIR']<=50,
#                                              final['New_FOIR']<=60,
#                                             final['New_FOIR']>60],choicelist=['<=35%','35-50%',
#                                                                                         '50-60%','60-70%'],default='None')

scrub_data['Time_on_Bureau_bkt'] = np.select(condlist=[scrub_data['months_since_oldest_trade']<=12,
                                                scrub_data['months_since_oldest_trade']<=18,
                                             scrub_data['months_since_oldest_trade']<=24,
                                             scrub_data['months_since_oldest_trade']<=36,
                                            scrub_data['months_since_oldest_trade']<=60,
                                            scrub_data['months_since_oldest_trade']<=84,
                                            scrub_data['months_since_oldest_trade']>84],
                                    choicelist=['<=12Mths','12-18Mths','19-24Mths','2-3 Yrs','3-5 Yrs','5-7 Yrs','7+ Yrs'],
                                    default='None')

scrub_data['cust_category'] = np.select(condlist=[((scrub_data['no_of_open_cc_trades']>=1) & (scrub_data['total_cl_of_cc_trades_l12m']>=50000))&((scrub_data['total_cl_open_auto_trades_l12m']>=200000)|(scrub_data['total_cl_open_mortgage_trades_l12m']>=1000000)),\
                                          ((scrub_data['no_of_open_cc_trades']>=1) & (scrub_data['total_cl_of_cc_trades_l12m']>=50000)),
                                          ((scrub_data['no_of_open_cc_trades']<=0) & (scrub_data['Unsecured High Credit Sum']>=50000))
                                            ],choicelist=['Carded 50K+ With Secured Loans','Carded 50K+',\
                                                          'Unsecured Loan 50K+'],
                                    default='None')

scrub_data['cust_hirisk'] = np.select(condlist=[((scrub_data['Unsecured High Credit Sum']<=82000) &
                                            (scrub_data['no_of_deduped_inquiries_l1m']>=2)&
                                           (scrub_data['no_of_trades_pl_2w_gl_cd']>=3))
                                            ],choicelist=[1],default=0)

# final['niro_user_id'] = final['user_id'].astype(str).str.strip()


In [ ]:
scrub_data.columns

In [ ]:
def preprocessor(df):

    try:
        df['CC_INACTIVE'] = np.select(condlist=[(df['bank_card_payment_category']=='INACTIVE')],choicelist=[1],default=0)
        df['CC_NOBC'] = np.select(condlist=[(df['bank_card_payment_category']=='NOBC')],choicelist=[1],default=0)
        df['CC_REVOLVER'] = np.select(condlist=[(df['bank_card_payment_category']=='REVOLVER')],choicelist=[1],default=0)
        df['CC_RVLRPLUS'] = np.select(condlist=[(df['bank_card_payment_category']=='RVLRPLUS')],choicelist=[1],default=0)
        df['CC_TRANSACTOR'] = np.select(condlist=[(df['bank_card_payment_category']=='TRANSACTOR')],choicelist=[1],default=0)
        df['CC_TRANPLUS'] = np.select(condlist=[(df['bank_card_payment_category']=='TRANPLUS')],choicelist=[1],default=0)
    except:
        print("errors")

    df['credithungry'] = np.select(condlist=[((df['UNSECURED_HIGH_CREDIT_SUM']<=75000)&(df['SECURED_HIGH_CREDIT_SUM']<=0)&\
                                            (df['no_of_deduped_inquiries_l6m']>=4))],choicelist=[1],default=0)

    df['no_of_open_cc_trades_rnkXno_of_business_general_trades_rnkXno_of_90p_accs_ever_rnkX3091']=np.select(condlist=[((df['no_of_open_cc_trades']>=2.0)&(df['no_of_open_cc_trades']<=21.0)&(df['no_of_business_general_trades']>=0.0)&(df['no_of_business_general_trades']<=0.0)&(df['no_of_90p_accs_ever']>=1.0)&(df['no_of_90p_accs_ever']<=11.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_cc_util_revolving_l1m_6020']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>39.22)&(df['max_aggregate_bankcard_utilisation_l12m']<=80.38)&(df['cc_util_revolving_l1m']>9.21)&(df['cc_util_revolving_l1m']<=538.69))],choicelist=[1],default=0)
    df['no_of_trades_pl_2w_gl_cd_worst_rating_l24m__unsecured_high_credit_sum_42']=np.select(condlist=[((df['no_of_trades_pl_2w_gl_cd']>10)&(df['no_of_trades_pl_2w_gl_cd']<=284)&(df['worst_rating_l24m']>-0.01)&(df['worst_rating_l24m']<=1)&(df['UNSECURED_HIGH_CREDIT_SUM']>520000)&(df['UNSECURED_HIGH_CREDIT_SUM']<=57875849))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXworst_rating_l3m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkX150']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=0.0)&(df['max_aggregate_bankcard_utilisation_l12m']<=34.5)&(df['worst_rating_l3m']>=1.0)&(df['worst_rating_l3m']<=1.0)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']>=10329.0)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']<=33080853.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_unsecured_high_credit_sum_3292']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>-6.01)&(df['max_aggregate_bankcard_utilisation_l12m']<=-1)&(df['UNSECURED_HIGH_CREDIT_SUM']>574040.33)&(df['UNSECURED_HIGH_CREDIT_SUM']<=1418204.33))],choicelist=[1],default=0)
    df['no_of_trades_total_cl_of_cc_trades_l12m_9968']=np.select(condlist=[((df['no_of_trades']>-0.01)&(df['no_of_trades']<=4)&(df['total_cl_of_cc_trades_l12m']>-5.01)&(df['total_cl_of_cc_trades_l12m']<=-1))],choicelist=[1],default=0)
    df['worst_rating_l3m_no_of_cc_trades_60P_l12m_7369']=np.select(condlist=[((df['worst_rating_l3m']>-0.01)&(df['worst_rating_l3m']<=1)&(df['no_of_cc_trades_60P_l12m']>-1.01)&(df['no_of_cc_trades_60P_l12m']<=0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkXsecured_balances_sum_rnkX447']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=38.85)&(df['max_aggregate_bankcard_utilisation_l12m']<=1027.37)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']>=-1.0)&(df['total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades']<=-1.0)&(df['SECURED_BALANCES_SUM']>=71607.0)&(df['SECURED_BALANCES_SUM']<=495403790.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_auto_trades_l12m_rnkXno_of_deduped_inquiries_l3m_rnkX6945']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=0.0)&(df['max_aggregate_bankcard_utilisation_l12m']<=38.84)&(df['total_bal_open_auto_trades_l12m']>=-3.0)&(df['total_bal_open_auto_trades_l12m']<=-1.0)&(df['no_of_deduped_inquiries_l3m']>=2.0)&(df['no_of_deduped_inquiries_l3m']<=2.0))],choicelist=[1],default=0)
    df['worst_rating_l24m_no_pl_sanctionedamt_lt30K_l36m_7642']=np.select(condlist=[((df['worst_rating_l24m']>1)&(df['worst_rating_l24m']<=1.5)&(df['no_pl_sanctionedamt_lt30K_l36m']>0)&(df['no_pl_sanctionedamt_lt30K_l36m']<=1))],choicelist=[1],default=0)
    df['worst_rating_l24m_no_of_cc_lt30p_l12m_541']=np.select(condlist=[((df['worst_rating_l24m']>1.5)&(df['worst_rating_l24m']<=9)&(df['no_of_cc_lt30p_l12m']>1)&(df['no_of_cc_lt30p_l12m']<=2))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_rnkXno_of_deduped_inquiries_l12m_rnkXno_of_30P_cc_l12m_rnkX6968']=np.select(condlist=[((df['no_of_60p_accs_ever']>=0.0)&(df['no_of_60p_accs_ever']<=0.0)&(df['no_of_deduped_inquiries_l12m']>=8.0)&(df['no_of_deduped_inquiries_l12m']<=73.0)&(df['no_of_30P_cc_l12m']>=1.0)&(df['no_of_30P_cc_l12m']<=3.0))],choicelist=[1],default=0)
    df['worst_rating_l3m_no_of_90p_accs_ever_6844']=np.select(condlist=[((df['worst_rating_l3m']>1)&(df['worst_rating_l3m']<=9)&(df['no_of_90p_accs_ever']>0)&(df['no_of_90p_accs_ever']<=16))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_no_of_30P_cc_l12m_8953']=np.select(condlist=[((df['no_of_60p_accs_ever']>-6.01)&(df['no_of_60p_accs_ever']<=0)&(df['no_of_30P_cc_l12m']>-1)&(df['no_of_30P_cc_l12m']<=0))],choicelist=[1],default=0)
    df['current_utilization_of_top_wallet_bankcard8'] = np.select(condlist=[((df['current_utilization_of_top_wallet_bankcard']>88)&(df['current_utilization_of_top_wallet_bankcard']<=600))],choicelist=[1],default=0)
    df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m'] = np.select(condlist=[df['total_bal_open_trades_l12m']<=0],choicelist=[df['no_of_deduped_inquiries']],default=df['no_of_deduped_inquiries']/df['total_bal_open_trades_l12m'])
    df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m'] = np.select(condlist=[df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m']>0.0156,df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m']<0.00000001],\
                                                        choicelist=[0.0156,0.00000001],default=df['no_of_deduped_inquiries_by_total_bal_open_trades_l12m'])

    df['cc_util_revolving_l1m_unsecured_high_credit_sum_6330']=np.select(condlist=[((df['cc_util_revolving_l1m']>-6.01)&(df['cc_util_revolving_l1m']<=-1)&(df['UNSECURED_HIGH_CREDIT_SUM']>-0.01)&(df['UNSECURED_HIGH_CREDIT_SUM']<=46297.17))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_rnkXno_of_open_cc_trades_rnkXno_tr_opened_l3m_rnkX1196']=np.select(condlist=[((df['no_of_60p_accs_ever']>=0.0)&(df['no_of_60p_accs_ever']<=0.0)&(df['no_of_open_cc_trades']>=0.0)&(df['no_of_open_cc_trades']<=1.0)&(df['no_tr_opened_l3m']>=2.0)&(df['no_tr_opened_l3m']<=22.0))],choicelist=[1],default=0)
    df['total_bal_open_trades_l12m_no_of_trades_pl_2w_gl_cd_4682']=np.select(condlist=[((df['total_bal_open_trades_l12m']>-5.01)&(df['total_bal_open_trades_l12m']<=28475.83)&(df['no_of_trades_pl_2w_gl_cd']>2)&(df['no_of_trades_pl_2w_gl_cd']<=4))],choicelist=[1],default=0)
    df['cc_util_revolving_l1m_rnkXno_of_deduped_inquiries_l3m_rnkXunsecured_high_credit_sum_rnkX8903']=np.select(condlist=[((df['cc_util_revolving_l1m']>=3.66)&(df['cc_util_revolving_l1m']<=148.68)&(df['no_of_deduped_inquiries_l3m']>=3.0)&(df['no_of_deduped_inquiries_l3m']<=31.0)&(df['UNSECURED_HIGH_CREDIT_SUM']>=123090.0)&(df['UNSECURED_HIGH_CREDIT_SUM']<=565217.0))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_no_of_deduped_inquiries_www']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>-6.01)&(df['max_aggregate_bankcard_utilisation_l12m']<=-2)&(df['no_of_deduped_inquiries']>6)&(df['no_of_deduped_inquiries']<=8))],choicelist=[1],default=0)
    df['months_since_oldest_trade_no_of_deduped_inquiries_l1m']=np.select(condlist=[((df['months_since_oldest_trade']>-5.01)&(df['months_since_oldest_trade']<=24)&(df['no_of_deduped_inquiries_l1m']>-0.01)&(df['no_of_deduped_inquiries_l1m']<=1))],choicelist=[1],default=0)
    df['no_of_90p_accs_ever_no_of_30P_cc_l24m_3957']=np.select(condlist=[((df['no_of_90p_accs_ever']>0)&(df['no_of_90p_accs_ever']<=16)&(df['no_of_30P_cc_l24m']>0)&(df['no_of_30P_cc_l24m']<=27))],choicelist=[1],default=0)
    df['total_bal_open_trades_l12m_worst_rating_l12m']=np.select(condlist=[((df['total_bal_open_trades_l12m']>72909.14)&(df['total_bal_open_trades_l12m']<=145061.71)&(df['worst_rating_l12m']>1)&(df['worst_rating_l12m']<=1.5))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXno_of_business_general_trades_rnkXworst_rating_l3m_rnkX571']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=42.59)&(df['max_aggregate_bankcard_utilisation_l12m']<=113.37)&(df['no_of_business_general_trades']>=1.0)&(df['no_of_business_general_trades']<=9.0)&(df['worst_rating_l3m']>=1.5)&(df['worst_rating_l3m']<=9.0))],choicelist=[1],default=0)
    df['no_of_trades_no_of_trades_pl_2w_gl_cd_8980']=np.select(condlist=[((df['no_of_trades']>24)&(df['no_of_trades']<=380)&(df['no_of_trades_pl_2w_gl_cd']>2)&(df['no_of_trades_pl_2w_gl_cd']<=4))],choicelist=[1],default=0)
    df['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_cl_open_pl_trades_l12m_rnkXworst_rating_l3m_rnkX2111']=np.select(condlist=[((df['max_aggregate_bankcard_utilisation_l12m']>=39.1)&(df['max_aggregate_bankcard_utilisation_l12m']<=178.39)&(df['total_cl_open_pl_trades_l12m']>=3600.0)&(df['total_cl_open_pl_trades_l12m']<=200000.0)&(df['worst_rating_l3m']>=1.5)&(df['worst_rating_l3m']<=9.0))],choicelist=[1],default=0)
    df['no_of_60p_accs_ever_no_of_deduped_inquiries_l12m_4']=np.select(condlist=[((df['no_of_60p_accs_ever']>-6.01)&(df['no_of_60p_accs_ever']<=0)&(df['no_of_deduped_inquiries_l12m']>5)&(df['no_of_deduped_inquiries_l12m']<=7))],choicelist=[1],default=0)
    df['worst_rating_l24m_cc_util_revolving_l1m_1387']=np.select(condlist=[((df['worst_rating_l24m']>1.5)&(df['worst_rating_l24m']<=9)&(df['cc_util_revolving_l1m']>-6.01)&(df['cc_util_revolving_l1m']<=-1))],choicelist=[1],default=0)
    df['no_of_trades_total_bal_of_open_pl_l12m_6726']=np.select(condlist=[((df['no_of_trades']>15)&(df['no_of_trades']<=24)&(df['total_bal_of_open_pl_l12m']>247.67)&(df['total_bal_of_open_pl_l12m']<=51449))],choicelist=[1],default=0)
    df['no_of_open_cc_trades_total_bal_open_trades_l12m_5298']=np.select(condlist=[((df['no_of_open_cc_trades']>-1)&(df['no_of_open_cc_trades']<=1)&(df['total_bal_open_trades_l12m']>-5.01)&(df['total_bal_open_trades_l12m']<=28475.83))],choicelist=[1],default=0)
    return df

def getNapsScore(row):
    
    zz = (-1.7249)+(row['no_of_60p_accs_ever_no_of_30P_cc_l12m_8953']*-0.6572)+(row['worst_rating_l6m']*-0.3441)+(row['worst_rating_l24m_cc_util_revolving_l1m_1387']*0.8992)+\
         (row['no_of_open_cc_trades_rnkXno_of_business_general_trades_rnkXno_of_90p_accs_ever_rnkX3091']*1.0623)+(row['current_utilization_of_top_wallet_bankcard8']*0.6604)+(row['no_of_deduped_inquiries_l6m']*0.0361)+\
         (row['max_aggregate_bankcard_utilisation_l12m_cc_util_revolving_l1m_6020']*-0.8515)+(row['no_of_deduped_inquiries_by_total_bal_open_trades_l12m']*31.4963)+(row['no_of_60p_accs_ever_no_of_deduped_inquiries_l12m_4']*-0.452)+\
         (row['no_of_trades_pl_2w_gl_cd_worst_rating_l24m__unsecured_high_credit_sum_42']*-0.7234)+(row['cc_util_revolving_l1m_unsecured_high_credit_sum_6330']*0.3369)+\
         (row['max_aggregate_bankcard_utilisation_l12m_rnkXworst_rating_l3m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkX150']*2.9448)+(row['no_of_60p_accs_ever_rnkXno_of_open_cc_trades_rnkXno_tr_opened_l3m_rnkX1196']*0.6413)+\
         (row['max_aggregate_bankcard_utilisation_l12m_unsecured_high_credit_sum_3292']*-0.7401)+(row['total_bal_open_trades_l12m_no_of_trades_pl_2w_gl_cd_4682']*-0.5306)+(row['no_of_trades_total_bal_of_open_pl_l12m_6726']*-1.0659)+\
         (row['no_of_trades_total_cl_of_cc_trades_l12m_9968']*0.3131)+(row['cc_util_revolving_l1m_rnkXno_of_deduped_inquiries_l3m_rnkXunsecured_high_credit_sum_rnkX8903']*1.3062)+\
         (row['worst_rating_l3m_no_of_cc_trades_60P_l12m_7369']*-0.6157)+(row['max_aggregate_bankcard_utilisation_l12m_no_of_deduped_inquiries_www']*0.8942)+(row['no_of_open_cc_trades_total_bal_open_trades_l12m_5298']*0.5325)+\
         (row['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades_rnkXsecured_balances_sum_rnkX447']*-0.4146)+(row['months_since_oldest_trade_no_of_deduped_inquiries_l1m']*0.2631)+\
         (row['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_bal_open_auto_trades_l12m_rnkXno_of_deduped_inquiries_l3m_rnkX6945']*-1.5711)+(row['no_of_90p_accs_ever_no_of_30P_cc_l24m_3957']*-1.1005)+\
         (row['worst_rating_l24m_no_pl_sanctionedamt_lt30K_l36m_7642']*0.472)+(row['total_bal_open_trades_l12m_worst_rating_l12m']*0.6542)+(\
             row['worst_rating_l24m_no_of_cc_lt30p_l12m_541']*0.8891)+(row['max_aggregate_bankcard_utilisation_l12m_rnkXno_of_business_general_trades_rnkXworst_rating_l3m_rnkX571']*1.6077)+\
             (row['no_of_60p_accs_ever_rnkXno_of_deduped_inquiries_l12m_rnkXno_of_30P_cc_l12m_rnkX6968']*1.6329)+(row['no_of_trades_no_of_trades_pl_2w_gl_cd_8980']*1.9295)+\
             (row['worst_rating_l3m_no_of_90p_accs_ever_6844']*0.9481)+(row['max_aggregate_bankcard_utilisation_l12m_rnkXtotal_cl_open_pl_trades_l12m_rnkXworst_rating_l3m_rnkX2111']*1.0015)

    r = np.exp(zz)/(1+np.exp(zz))
    scr = int(max(350,min(575*(r**(-0.08)),900)))

    return scr

scrub_data['SECURED_ACCOUNTS_COUNT']=scrub_data['Secured Accounts Count']
scrub_data['UNSECURED_ACCOUNTS_COUNT']=scrub_data['Unsecured Accounts Count']
scrub_data['SECURED_HIGH_CREDIT_SUM']=scrub_data['Secured High Credit Sum']
scrub_data['UNSECURED_HIGH_CREDIT_SUM']=scrub_data['Unsecured High Credit Sum']
scrub_data['SECURED_AMOUNT_OVERDUE_SUM']=scrub_data['Secured Amount Overdue Sum']
scrub_data['UNSECURED_AMOUNT_OVERDUE_SUM']=scrub_data['Unsecured Amount Overdue Sum']
scrub_data['SECURED_BALANCES_SUM']=scrub_data['Secured Balances Sum']
scrub_data['UNSECURED_BALANCES_SUM']=scrub_data['Unsecured Balances Sum']
              
scrub_data = preprocessor(scrub_data)
scrub_data['naps_new']= scrub_data.apply(getNapsScore,axis=1)
scrub_data['naps_new_bkt'] = scrub_data.apply(naps_score_bkt,var='naps_new',axis=1)

In [ ]:
cols = ['Member Reference', 'Mobile_No',
       'max_aggregate_bankcard_utilisation_l12m', 'cc_util_revolving_l1m',
       'bank_card_payment_category', 'no_of_60p_accs_ever',
       'no_of_deduped_inquiries', 'total_cl_open_mortgage_trades_l12m',
       'total_balance_open_mortgage_trades_l12m', 'total_bal_of_open_pl_l12m',
       'months_since_oldest_trade', 'no_of_mortgage_trades',
       'no_of_open_cc_trades', 'no_of_business_general_trades',
       'no_of_30p_accs_ever', 'no_of_trades', 'total_bal_open_trades_l12m',
       'total_bal_open_auto_trades_l12m', 'months_since_recent_chargedoff180P',
       'total_cl_open_auto_trades_l12m', 'total_cl_open_pl_trades_l12m',
       'no_of_chargedoff180P', 'total_cl_of_cc_trades_l12m',
       'no_of_90p_accs_ever', 'total_bal_of_chargedoff_trades',
       'worst_rating_l12m', 'current_utilization_of_top_wallet_bankcard',
       'no_of_trades_pl_2w_gl_cd',
       'total_bal_open_trades_l12m_except_wo_cc_od',
       'no_of_deduped_inquiries_l12m', 'no_of_deduped_inquiries_l6m',
       'no_of_deduped_inquiries_l3m', 'no_of_deduped_inquiries_l1m',
       'worst_rating_l24m', 'worst_rating_l6m', 'worst_rating_l3m',
       'worst_rating_l1m', 'no_of_60p_accs_l24m', 'no_of_60p_accs_l12m',
       'no_of_trades_bounced_l36m', 'no_of_trades_bounced_l12m',
       'no_of_cc_trades_60P_l24m', 'no_of_30P_cc_l24m',
       'no_of_cc_trades_60P_l12m', 'no_of_30P_cc_l12m', 'no_of_cc_lt30p_l12m',
       'no_cc_sanctionedamt_lt30K_l36m', 'no_pl_sanctionedamt_lt30K_l36m',
       'no_tr_opened_l6m', 'worst_dpd_l36m',
       'total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades',
       'no_tr_opened_l12m', 'no_tr_opened_l3m', 'per_of_30p_accs_ever',
       'total_payment_amount_bankcard_accs_l3m',
       'max_revolving_monthly_spend_l12m', 'total_monthly_obligations_l3m',
       'total_balance_bankcard_accs_l3m', 'no_of_trades_unsec',
       'total_payment_amount_l3m', 'worst_dpd_l2m', 'Secured Accounts Count',
       'Unsecured Accounts Count', 'Secured High Credit Sum',
       'Unsecured High Credit Sum', 'Secured Amount Overdue Sum',
       'Unsecured Amount Overdue Sum', 'Secured Balances Sum',
       'Unsecured Balances Sum', 'Own Accounts count', 'Other Accounts count',
       'CIBIL_score', 'CIBIL_score_bkt', 
       'no_of_open_cc_trades_bkt', 'Years on Bureau_bkt',
       'no_of_deduped_inquiries_bkt', 'Secured Loan_bkt', 'Time_on_Bureau_bkt',
       'cust_category', 'cust_hirisk', 'CC_INACTIVE', 'CC_NOBC', 'CC_REVOLVER',
       'CC_RVLRPLUS', 'CC_TRANSACTOR', 'CC_TRANPLUS', 'credithungry', 'naps_new', 'naps_new_bkt']

In [ ]:
scrub_data = scrub_data[cols]

In [ ]:
scrub_data.head()